# looking into SQL database
## CIA World Factbook, a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [3]:
%%sql
select * 
    FROM sqlite_master
    where type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [4]:
%%sql
select *
    from facts
    limit 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


name — the name of the country.

area— the country's total area (both land and water).

area_land — the country's land area in square kilometers.

area_water — the country's waterarea in square kilometers.

population — the country's population.

population_growth— the country's population growth as a percentage.

birth_rate — the country's birth rate, or the number of births per year per 1,000 people.

death_rate — the country's death rate, or the number of death per year per 1,000 people.

In [7]:
%%sql
select count(*) as 'total entries in the table'
    from facts;

Done.


total entries in the table
261


In [ ]:
## a single query that returns Minimum population, Maximum population, Minimum population growth ,Maximum population growth


In [9]:
%%sql
select min(population), max(population), min(population_growth), max(population_growth)
    from facts;

Done.


min(population),max(population),min(population_growth),max(population_growth)
0,7256490011,0.0,4.02


In [ ]:
# a query that returns the countries with the minimum population.

In [17]:
%%sql
select name, population
    from facts
    where population = (select min(population) from facts);

Done.


name,population
Antarctica,0


In [14]:
# a query that returns the countries with the maximum population.

In [16]:
%%sql
select name, population
    from facts
    where population = (select max(population) from facts);

Done.


name,population
World,7256490011


In [ ]:
# re run statistics exclding the world value

In [23]:
%%sql
select min(population), max(population), min(population_growth), max(population_growth)
    from facts
    where name not in ('World');

Done.


min(population),max(population),min(population_growth),max(population_growth)
0,1367485388,0.0,4.02


In [ ]:
# aside from the world value which country has the larget population

In [24]:
%%sql
select name, population
    from facts
    where population = (select max(population) from facts where name not in ('World'))

Done.


name,population
China,1367485388


In [26]:
%%sql 
select avg(population), avg(area)
    from facts
    where name not in ('World');

Done.


avg(population),avg(area)
32242666.56846473,555093.546184739


## Find countries that are densely populated, having the following:

Above-average values for population.
Below-average values for area.

In [27]:
%%sql
select name, population, area
    from facts
    where population > (select avg(population) from facts where name not in ('World'))
    and area < (select avg(area) from facts where name not in ('World'));

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Iraq,37056169,438317
Italy,61855120,301340
Japan,126919659,377915
"Korea, South",49115196,99720
Morocco,33322699,446550
Philippines,100998376,300000
Poland,38562189,312685
Spain,48146134,505370


## Which country has the highest growth rate?

In [30]:
%%sql
select name, population_growth
    from facts
    where population_growth = (select max(population_growth) from facts where name not in ('World') )

Done.


name,population_growth
South Sudan,4.02


## Which countries have the highest ratios of water to land? 

In [34]:
%%sql
select avg(cast(area_water as float) / area_land) as avg_area_water_to_land
    from facts
    where name not in ('World');

Done.


avg_area_water_to_land
3.803454624004138


In [39]:
%%sql
select name, area_water,area_land, cast(area_water as float) / area_land as area_water_to_land
    from facts
    where area_water_to_land> (select avg(cast(area_water as float) / area_land) as avg_area_water_to_land
                                    from facts
                                    where name not in ('World'));

Done.


name,area_water,area_land,area_water_to_land
British Indian Ocean Territory,54340,60,905.6666666666666
Virgin Islands,1564,346,4.520231213872832


## Which countries have a higher death rate than birth rate?

In [43]:
%%sql
select name, birth_rate, death_rate, death_rate-birth_rate as difference
    from facts
    where name != 'World'
    and difference>0;

Done.


name,birth_rate,death_rate,difference
Austria,9.41,9.42,0.009999999999999787
Belarus,10.7,13.36,2.66
Bosnia and Herzegovina,8.87,9.75,0.8800000000000008
Bulgaria,8.92,14.44,5.52
Croatia,9.45,12.18,2.7300000000000004
Czech Republic,9.63,10.34,0.7099999999999991
Estonia,10.51,12.4,1.8900000000000006
Germany,8.47,11.42,2.9499999999999993
Greece,8.66,11.09,2.4299999999999997
Hungary,9.16,12.73,3.5700000000000003
